<a href="https://colab.research.google.com/github/JanRusselCruz/SultiWag/blob/Source_Code/SultiWag_AudioToSpectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
%cd /content/gdrive/MyDrive/SultiWag/AudioFiles

In [ ]:
audio_path = "/content/gdrive/MyDrive/SultiWag/AudioFiles/03_ConsistentData_584/Kagan"
spectrogram_path = "/content/gdrive/MyDrive/SultiWag/AudioFiles/02_Spectrograms/Consistent584_Spectrograms"

In [ ]:
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import wave, os, glob
from tqdm import tqdm

In [ ]:
def load_files(path=""):
    files = []
    file_list = glob.glob(os.path.join(path, '*.mp3'))
    no_translation = []
    progress_bar = tqdm(file_list, desc="Loading files", position=0)
    for filename in progress_bar:
        try:
            file = filename.split("/")
            file = file[len(file) - 1].split(".")[0]
            term = file.split("_")
            english = term[0]
            translation = term[1] if len(term) > 1 else None
            x, sr = librosa.load(filename, sr=22050)
            if translation is None:
                no_translation.append(os.path.basename(filename))
                #os.remove(filename)
            else:
                files.append([x, sr, filename, english, translation])
            progress_bar.set_description(f"Loading {os.path.basename(filename)}")
        except Exception as e:
            print(f"Error loading {filename}: {e}")
    progress_bar.close()

    if no_translation:
        print("Files without translation:")
        for filename in no_translation:
            print(filename)

    return files


In [ ]:
files=[]
files=load_files(path=audio_path+"/all_voices")

In [ ]:
for file in tqdm(files):
    file_folders=file[2].split('/')
    filename=file_folders[len(file_folders)-1].split('.')[0]
    
    y, sr = librosa.load(file[2], sr=44100)
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    times = librosa.times_like(f0)
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)


    fig, ax = plt.subplots()
    img = librosa.display.specshow(D)    
    
    fig.savefig(spectrogram_path+"/"+filename+".jpg")
    # plt.show()
    plt.clf()
    
    # break
# plt.show()